In [ ]:
from ast import Break
import pandas as pd

In [ ]:
# Read TSV file into DataFrame

column = ["qid", "Q0", "docno", "rank", "score", "tag"]
df1 = pd.read_table('../Step_1_Individual_Models/LKAU23_run60_ArabicBERT.tsv', names=column)
# print(df1)


df2 = pd.read_table('../Step_1_Individual_Models/LKAU23_run60_CL_AraBERT.tsv', names=column)
# print(df2)

In [ ]:
# Get the matching passages from two files/dataframe
# Calculate the average score for each matching passages

All_qid=[]
All_Q0=[]
All_docno=[]
All_score=[]
All_tag=[]



sumscore=0
for index1, row1 in df1.iterrows():
  sumscore=0
  Transqi1= row1["qid"]
  Transdocno1= row1["docno"]

  for index2, row2 in df2.iterrows():
    Transqi2= row2["qid"]
    Transdocno2= row2["docno"]

    if Transqi1 == Transqi2 and Transdocno1 == Transdocno2:
        print(Transqi1, ' and ', Transqi2, 'with ', Transdocno1, ' and ', Transdocno2)
        sumscore = row1["score"] + row2["score"]
        sumscore = sumscore/float(2)
        All_qid.append(Transqi1)
        All_Q0.append("Q0")
        All_docno.append(Transdocno1)
        All_score.append(sumscore)
        All_tag.append("Ensemble_ArabicBERT_CL_AraBERT")
        print(row1["score"], ' and ', row2["score"])
        print(sumscore)
        break


In [ ]:
result = {"qid": All_qid,
          "Q0": All_Q0,
          "docno": All_docno,
          "score": All_score,
          "tag": All_tag}

dfresults = pd.DataFrame(result)
dfresults

In [ ]:
dfresults = dfresults.sort_values(by=['qid', 'score'], ascending=[True, False])
dfresults

In [ ]:
# We get the passages that are not in the ensemble DataFrame

qid_final = []
score_final = []
docno_final = []
q0_final = []
tag_final = []

for index1, row1 in df1.iterrows():

    df1qid1 = row1['qid']
    df1docno1 = row1['docno']
    df1score1 = row1['score']

    for index2, row2 in dfresults.iterrows():
        dfresqid = row2['qid']
        dfresdocno = row2['docno']
        dfresscore = row2['score']

        # Convert the passages (docno) into a list for checking based on (qid)
        df_qid = dfresults.loc[dfresults['qid'] == dfresqid]
        docno_list = df_qid['docno'].tolist()

        if df1qid1 == dfresqid and df1docno1 in docno_list:
            break
        elif df1qid1 == dfresqid and df1docno1 not in docno_list:
            df_2_qid = df2.loc[df2['qid'] == df1qid1]

            for index2, row2 in df_2_qid.iterrows():
                df1qid2 = row2['qid']
                df1docno2 = row2['docno']
                df1score2 = row2['score']

                if df1qid1 == df1qid2 and df1docno2 not in docno_list:
                    qid_final.append(df1qid1)
                    score_final.append(df1score1)
                    docno_final.append(df1docno1)
                    q0_final.append("Q0")
                    tag_final.append("ArabicBERT")

                    qid_final.append(df1qid2)
                    score_final.append(df1score2)
                    docno_final.append(df1docno2)
                    q0_final.append("Q0")
                    tag_final.append("CL-AraBERT")

In [ ]:
df_results_diff = pd.DataFrame({
    'qid': qid_final,
    'Q0': q0_final,
    'docno': docno_final,
    'score': score_final,
    'tag': tag_final
})

df_results_diff

In [ ]:
# Sort the passages based on the score

df_results_diff2 = df_results_diff.sort_values(by=['qid', 'score'], ascending=[True, False])
df_results_diff2

In [ ]:
# Delete the duplicates

df_results_diff3 = df_results_diff2.drop_duplicates(subset=['qid', 'docno'])
df_results_diff3.head(30)

In [ ]:
# Get the question IDs and store them into a list

df_test_set = pd.read_csv("../../quran-qa-2023-main/Task-A/data/QQA23_TaskA_test.tsv", sep='\t', names=['ID', 'Questions'])
questions_id = df_test_set.ID.tolist()
questions_id[:5]

[500, 502, 504, 505, 508]

In [ ]:
# Merge the matching passages along with other passages.
# Also we're putting the matching/ensembled passages at the top

for q in questions_id:
    if q == 500:
        df_temp = dfresults.loc[dfresults['qid'] == q]
        df_temp2 = df_results_diff3.loc[df_results_diff3['qid'] == q]
        df_final_results = pd.concat([df_temp, df_temp2])
    else:
        df_temp = dfresults.loc[dfresults['qid'] == q]
        df_temp2 = df_results_diff3.loc[df_results_diff3['qid'] == q]
        df_temp_results = pd.concat([df_temp, df_temp2])
        df_final_results = pd.concat([df_final_results, df_temp_results])


In [ ]:
df_final_results.head(35)

In [ ]:
df_final_results.reset_index(drop=True, inplace=True)
df_final_results.head(40)

In [ ]:
# Extract the first 10

qid2=[]
Q02=[]
docno2=[]
score2=[]
tag2=[]
rank2=[]

flage=df_final_results["qid"][0]
num=1
listate=[]
for index, row in df_final_results.iterrows():
  if flage != row["qid"]:
    num=1
    flage = row["qid"]
    qid2.append(row["qid"])
    Q02.append(row["Q0"])
    docno2.append(row["docno"])
    score2.append(row["score"])
    tag2.append(row["tag"])
    rank2.append(num)
  else:
    if num <= 10:
      qid2.append(row["qid"])
      Q02.append(row["Q0"])
      docno2.append(row["docno"])
      score2.append(row["score"])
      tag2.append(row["tag"])
      rank2.append(num)

  num=num+1

data2 = {"qid":qid2, "Q0":Q02, "docno":docno2, "rank":rank2, "score":score2, "tag":tag2}


df_final_results2 = pd.DataFrame(data2)
df_final_results2

In [ ]:
df_final_results2.to_csv("LKAU23_run61.tsv", sep='\t', index=False, header=False)

In [ ]:
! python ../../quran-qa-2023-main/Task-A/code/QQA23_TaskA_submission_checker.py \
    -m "LKAU23_run61.tsv"

Format check: Passed
